# Creating multilevel HEALPix Data in zarr and datatree for WW3 model 

In this notebook, we will demonstrate examples of saving multilevel HEALPix data from WW3 model output.  

## Setup

To run this notebook, you need to install XDGGS. You can find the xdggs repository here: [XDGGS GitHub Repository](https://github.com/xarray-contrib/xdggs.git).


1. You can either install `xdggs` with the necessary dependencies, use the following command:

```bash
pip install xdggs
```

2. You will need up to date xarray package :

```bash
pip install -U xarray
```

3. You can use zarr version zarr==2.18.4.  In a few weeks, it would work with icechunk as well. :

```bash
#pip install icechunk
pip install -U zarr==2.18.4
```


In [3]:
import warnings

import numpy as np
import xarray as xr
import xdggs

warnings.filterwarnings("ignore")

In [4]:
xr.__version__

'2024.11.0'

In [5]:
import zarr

zarr.__version__

'2.18.4'

##  Load npy data in  Healpix coordinate 
2048 does not work

In [6]:
file_path = "file_2048.npy"
data = np.load(file_path)
total_cells = len(data)
level = round(np.log(total_cells / 12) / np.log(4))

# Create cell_ids
cell_ids = np.arange(12 * 4**level)
# Define grid_info
grid_info = {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"}
# Create xarray.Dataset
ds = xr.Dataset(
    {"data": (("cells",), data)}, coords={"cell_ids": ("cells", cell_ids, grid_info)}
).pipe(xdggs.decode)
ds = ds.where(ds > 0, drop=True)
ds["data"].dggs.explore(cmap="viridis", alpha=0.8)

ValueError: cannot reshape array of size 42336240 into shape (50331648,)

In [16]:
import numpy as np
import xarray as xr
import xdggs  # Assuming xdggs is available

# Define file sizes and initialize the dictionary
file_sizes = [512, 1024]  # , 2048]
plevels = {}

# Process each file size
for file_size in file_sizes:
    print(file_size)
    # Construct the file name dynamically
    file_path = f"file_{file_size}.npy"

    # Load the .npy file
    data = np.load(file_path)

    # Calculate the level
    total_cells = len(data)
    level = round(np.log(total_cells / 12) / np.log(4))

    # Create cell_ids
    cell_ids = np.arange(12 * 4**level)

    # Define grid_info
    grid_info = {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"}

    # Create xarray.Dataset and apply filters
    ds = (
        xr.Dataset(
            {"data": (("cells",), data)},
            coords={"cell_ids": ("cells", cell_ids, grid_info)},
        )
        .pipe(xdggs.decode)
        .where(lambda d: d > 0, drop=True)
    )

    # Store in dictionary with level as key
    plevels[str(level)] = ds

# Print the dictionary keys to confirm
plevels

512
1024


{'9': <xarray.Dataset> Size: 41kB
 Dimensions:   (cells: 2561)
 Coordinates:
   * cell_ids  (cells) int64 20kB 174927 174931 174933 ... 906354 906496 906498
 Dimensions without coordinates: cells
 Data variables:
     data      (cells) float64 20kB 0.4569 0.3647 0.4264 ... 0.546 0.8065 0.8
 Indexes:
     cell_ids  HealpixIndex(nside=9, indexing_scheme=nested),
 '10': <xarray.Dataset> Size: 142kB
 Dimensions:   (cells: 8901)
 Coordinates:
   * cell_ids  (cells) int64 71kB 699708 699709 699710 ... 3625986 3625992
 Dimensions without coordinates: cells
 Data variables:
     data      (cells) float64 71kB 0.326 0.421 0.414 0.5363 ... 0.838 0.82 0.8
 Indexes:
     cell_ids  HealpixIndex(nside=10, indexing_scheme=nested)}

## Create a datatree including different levels of HEALPix data



In [8]:
# plevels = {"/": root, "9": ds, "8": ds_8, "7": ds_7}

In [17]:
ds_all = xr.DataTree.from_dict(plevels)
ds_all

<xarray.DataTree>
Group: /
├── Group: /9
│       Dimensions:   (cells: 2561)
│       Coordinates:
│         * cell_ids  (cells) int64 20kB 174927 174931 174933 ... 906354 906496 906498
│       Dimensions without coordinates: cells
│       Data variables:
│           data      (cells) float64 20kB 0.4569 0.3647 0.4264 ... 0.546 0.8065 0.8
│       Indexes:
│           cell_ids  HealpixIndex(nside=9, indexing_scheme=nested)
└── Group: /10
        Dimensions:   (cells: 8901)
        Coordinates:
          * cell_ids  (cells) int64 71kB 699708 699709 699710 ... 3625986 3625992
        Dimensions without coordinates: cells
        Data variables:
            data      (cells) float64 71kB 0.326 0.421 0.414 0.5363 ... 0.838 0.82 0.8
        Indexes:
            cell_ids  HealpixIndex(nside=10, indexing_scheme=nested)

## Save it to zarr


In [18]:
ds_all.to_zarr("ww3.zarr", mode="w")

### Load multigrid data from zarr and plot them together

In [15]:
data = xr.open_datatree("ww3.zarr")
data
data["9"]["data"].compute().pipe(xdggs.decode).dggs.explore(
    center=0, cmap="coolwarm", alpha=0.8
)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f1eec80…

In [9]:
import numpy as np


def create_arrow_table(polygons, arr, coords=None):
    from arro3.core import Array, ChunkedArray, Schema, Table

    if coords is None:
        coords = ["latitude", "longitude"]

    array = Array.from_arrow(polygons)
    name = arr.name or "data"
    arrow_arrays = {
        "geometry": array,
        "cell_ids": ChunkedArray([Array.from_numpy(arr.coords["cell_ids"])]),
        name: ChunkedArray([Array.from_numpy(arr.data)]),
    } | {
        coord: ChunkedArray([Array.from_numpy(arr.coords[coord].data)])
        for coord in coords
        if coord in arr.coords
    }

    fields = [array.field.with_name(name) for name, array in arrow_arrays.items()]
    schema = Schema(fields)

    return Table.from_arrays(list(arrow_arrays.values()), schema=schema)


def normalize(var, center=None):
    from matplotlib.colors import CenteredNorm, Normalize

    if center is None:
        vmin = var.min(skipna=True)
        vmax = var.max(skipna=True)
        normalizer = Normalize(vmin=vmin, vmax=vmax)
    else:
        halfrange = np.abs(var - center).max(skipna=True)
        normalizer = CenteredNorm(vcenter=center, halfrange=halfrange)

    return normalizer(var.data)


def exploire_layer(
    arr,
    cell_dim="cells",
    cmap="viridis",
    center=None,
    alpha=None,
):
    from lonboard import SolidPolygonLayer
    from lonboard.colormap import apply_continuous_cmap
    from matplotlib import colormaps

    if len(arr.dims) != 1 or cell_dim not in arr.dims:
        raise ValueError(
            f"exploration only works with a single dimension ('{cell_dim}')"
        )

    cell_ids = arr.dggs.coord.data
    grid_info = arr.dggs.grid_info

    polygons = grid_info.cell_boundaries(cell_ids, backend="geoarrow")

    normalized_data = normalize(arr.variable, center=center)

    colormap = colormaps[cmap]
    colors = apply_continuous_cmap(normalized_data, colormap, alpha=alpha)

    table = create_arrow_table(polygons, arr)
    layer = SolidPolygonLayer(table=table, filled=True, get_fill_color=colors)

    return layer

In [14]:
import lonboard

arr9 = (
    data["9"]
    .ds["data"]
    .pipe(xdggs.decode)  # .dggs.sel_latlon(latitude=48, longitude=0)["data"]
)
arr10 = (
    data["10"]
    .ds["data"]
    .pipe(xdggs.decode)  # .dggs.sel_latlon(latitude=49, longitude=0)["data"]
)
# cmap="Reds"
lonboard.Map(
    [
        exploire_layer(
            arr9,
            alpha=1,
        ),
        exploire_layer(arr10, alpha=1),
    ]
)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f1eec80…

## Whats next

Use WW3 unstructured grid originated data